# Sarima(0,1,1)(0,1,1), un modelo de tendencia que varía con el tiempo

Esta vez se realizará un proceso de estimación para un modelo Arima que asume una media variable en el tiempo. De nueva cuenta se utiliza y tratan las series de conteos en el tiempo obtenidos de Ecobici. La descripción de los datos con los que se entrena es la siguiente:

- Se remueven fines de semana, haciendo solo uso de los días entre semana.
- Se eliminan los viajes realizados con la etiqueta 'C' (cancelados) en los logs de viajes.
- Se remueven los horarios de 0:00 a 5:00 horas debido a que el sistema no ofrece servicio en ese tiempo.
- Con la finalidad de obtener estabilidad en la actividad de los viajes, el análisis se comienza desde 01-03-2010.
- El siguiente análisis considera solo las estaciones que conforman la primera fase del sistema.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns; sns.set()
import datetime as dt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 18, 10

In [2]:
import statsmodels.api as sm
import rpy2.robjects as R
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [3]:
k_3 = pd.read_csv(r'cluster_member3.csv')
res = pd.read_csv(r'phase1_no_normalized.csv').T

res['cluster'] = k_3.values
temp_res = res.sort_values(['cluster'])

In [4]:
path = "ecobici.csv"
trips = pd.read_csv(path,
                    parse_dates=['date_removed', 'date_arrived'],
                    infer_datetime_format=True,
                    low_memory=False)

# Find the start date
ind = pd.DatetimeIndex(trips.date_arrived)
trips['date'] = ind.date.astype('datetime64')
trips['hour'] = ind.hour

trips = trips.loc[trips['action'] != 'C  ']

idx = pd.date_range(trips.date.min(), trips.date.max() + dt.timedelta(days=1), freq='H')

by_hour = (trips
     .set_index('date_arrived')
     .groupby([pd.TimeGrouper('H'), 'station_arrived'])
     .station_arrived
     .count()
     .unstack()
     .reindex(idx)
     .fillna(0)
     )
# Delete activity of non existent station's IDs in the dataset
by_hour.drop([col for col in list(by_hour.columns.values) if col > 275], axis=1, inplace=True)

# Keep only weekdays to make a more stable count signal
by_hour = by_hour[by_hour.index.dayofweek < 5]

# Since the system's opening, the slow activity of the first days were removed
by_hour = by_hour['2010-03-01':]

# Keep business hours for the system. Ecobici closes from 0:00 to 5:00
by_hour = by_hour.between_time('6:00','23:00')

Además de tratamiento antes mencionado, las series de tiempo fue sometida a un escalamiento de logaritmo, no sin antes adicionar 1 a cada conteo de las estaciones para evitar realizar logartimos sobre valores donde no está definido.

*Nota: Los datos con los que se realiza la regresión estan en escala log, solo es cuestión de realizar la operación inversa para obtener los resultados reales (aplicar exp y después restar 1). Para fines de observación se precindió de esta operación.

In [5]:
logged_ts = by_hour + 1
logged_ts = logged_ts.apply(np.log)

# deal with missing values. see issue
logged_ts.interpolate(inplace=True)

In [6]:
head = temp_res[temp_res.cluster == 1]
body = temp_res[temp_res.cluster == 2].sample(10)
tail = temp_res[temp_res.cluster == 3]

sub_sample = pd.concat([head,body,tail])

# Sección para las predicciones

In [7]:
#80-20 Train & test

test = logged_ts['2013-11-29':'2013-12-02']
train = logged_ts[:'2013-11-28']

In [8]:
%load_ext rpy2.ipython

In [9]:
stats = importr('stats')
tseries = importr('tseries')
forecast = importr('forecast')

In [10]:
from sklearn.metrics import mean_absolute_error

In [11]:
order = R.IntVector((0,1,1))
season = R.ListVector({'order': R.IntVector((0,1,1)), 'period' : 18})

columns = ['AIC','Durbin-Watson','RSS train','Unit root','Mean Absolute Error']
df_ = pd.DataFrame(index = sub_sample.index.astype(int),columns=columns)
df_ = df_.fillna(0) # with 0s rather than NaNs

forecast_matrix = pd.DataFrame(index = test.index, columns=sub_sample.index.astype(int))
forecast_matrix = forecast_matrix.fillna(0)
    
arima_models = []

for i in sub_sample.index.astype(int):
    r_df = pandas2ri.py2ri(train[[i]])
    y = stats.ts(r_df)
    model = stats.arima(y, order = order, seasonal=season)
    
    %Rpush model
    AIC = R.r('model$aic')[0]
    durbin_watson = sm.stats.durbin_watson(R.r('model$residuals'))
    RSS = sum((train[[i]].values-R.r('fitted(model)'))**2)
    unit_root = sum(R.r('model$coef'))
    
    f = forecast.forecast(model,len(test))
    pred = [j[1] for j in f[3].items()]
    dt = test.index
    pr = pd.Series(pred, index = dt)
    mae = mean_absolute_error(test[i],pr)
    forecast_matrix[i] = pred

    df_.loc[i] = [AIC, durbin_watson, RSS, unit_root, mae]
    arima_models.append(model)

In [12]:
df_

,AIC,Durbin-Watson,RSS train,Unit root,Mean Absolute Error
1,22746.920937,1.735348,3741.412657,-1.824824,0.226714
64,22007.940518,1.710850,3586.999463,-1.833326,0.240286
27,26147.172072,1.679405,4540.226369,-1.752635,0.321476
36,20774.375492,1.830261,3345.933070,-1.771760,0.254638
13,28053.811516,1.710376,5057.420590,-1.784436,0.308225
80,27260.347531,1.753486,4833.212626,-1.877506,0.658121
44,26159.904463,1.846570,4541.412971,-1.883618,0.396992
60,25812.354376,1.802945,4451.865733,-1.911725,0.248597
59,25012.243414,1.804359,4254.257523,-1.875230,0.260124
62,25617.103931,1.884368,4403.424673,-1.884623,0.321655


Los resultados en la tabla muestran que los coeficientes en el modelo no tienen una raiz unitaria, lo cual dice que los parámetros son suficientemente representativos para describir la señal. Por otro lado los valores de error MAE entre head body y tail no son demasiado diferentes entre si.

# Predicción para el conjunto de prueba

Los siguiente gráficos muestran los resultados obtenidos por la predicción del modelo contra los valores reales en el conjunto de prueba:

In [13]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

### Predicciones para head

In [14]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[1], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[1], color='red', alpha=0.5)


show(p)

In [15]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[64], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[64], color='red', alpha=0.5)

show(p)

### Predicciones en body 

In [16]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[59], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[59], color='red', alpha=0.5)

show(p)

In [17]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[60], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[60], color='red', alpha=0.5)

show(p)

### Predicciones en tail 

In [18]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[88], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[88], color='red', alpha=0.5)

show(p)

In [19]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[89], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[89], color='red', alpha=0.5)

show(p)

### ¿porqué no usar periodos de tiempo más largos para el conjunto de prueba?

El modelo $Arima(0,1,1)(0,1,1)_{12}$ es un proceso que asume una tendencia no constante, que varía con los valores locales, estos tipos de modelos generalmente tienen intervalos de confianza más amplios para predicciones a corto plazo

$$\hat{Y} = Y_{t-18} + Y_{t-1} - Y_{t-19} - \theta _1e_{t-1} - \Theta _1 e_{t-18} + \theta _1 \Theta_1 e_{t-19}$$

Donde $θ_1$ es un coeficiente MA(1) y $Θ_1$ es el coeficiente SMA(1). 

Los modelos que no asumen una tendencia variante con el tiempo generalmente tienen intervalos de confianza más angostos para las predicciones a largo plazo, pero eso no significa que lo haga mejor modelo a no ser que la suposición sea correcta:

$$\hat{Y} = \mu + Y_{t-18} + \phi _1 (Y_{t-1} - Y_{t-19})$$

Aquí $\phi_1$ denota el coeficiente AR(1)

El término adicional de lado derecho de la ecuación es un múltiplo de la diferencia estacional observada el día pasado, el cual tiene el efecto de corregir la predicción efecto de una hora con actividad inusual. $\mu$ representa la constante en la ecuación de predicción, el valor medio de estacionalidad en la serie diferenciada que es la tendencia diaria en las predicciones a largo plazo de este modelo. La constante es por definición igual a la media multiplicada por 1 menos el coeficiente AR(1).

# ¿Qué pasa si de todas maneras lo hacemos?

In [20]:
#80-20 Train & test
start = (len(logged_ts) * 80) / 100

train = logged_ts.iloc[:start]
test = logged_ts.iloc[start:]

In [21]:
columns = ['AIC','Durbin-Watson','RSS train','Unit root','Mean Absolute Error']
df_ = pd.DataFrame(index = sub_sample.index.astype(int),columns=columns)
df_ = df_.fillna(0) # with 0s rather than NaNs

forecast_matrix = pd.DataFrame(index = test.index, columns=sub_sample.index.astype(int))
forecast_matrix = forecast_matrix.fillna(0)
    
arima_models = []

for i in sub_sample.index.astype(int):
    r_df = pandas2ri.py2ri(train[[i]])
    y = stats.ts(r_df)
    model = stats.arima(y, order = order, seasonal=season)
    
    %Rpush model
    AIC = R.r('model$aic')[0]
    durbin_watson = sm.stats.durbin_watson(R.r('model$residuals'))
    RSS = sum((train[[i]].values-R.r('fitted(model)'))**2)
    unit_root = sum(R.r('model$coef'))
    
    f = forecast.forecast(model,len(test))
    pred = [j[1] for j in f[3].items()]
    dt = test.index
    pr = pd.Series(pred, index = dt)
    mae = mean_absolute_error(test[i],pr)
    forecast_matrix[i] = pred

    df_.loc[i] = [AIC, durbin_watson, RSS, unit_root, mae]
    arima_models.append(model)

In [22]:
df_

,AIC,Durbin-Watson,RSS train,Unit root,Mean Absolute Error
1,19143.828277,1.746690,3174.057508,-1.836544,0.436260
64,18233.215277,1.722757,2979.355664,-1.833655,0.432956
27,20847.811673,1.725149,3574.543732,-1.757360,0.426635
36,17285.898137,1.837995,2790.782810,-1.814516,0.956884
13,23062.041846,1.716472,4165.602508,-1.784239,0.664743
80,22867.456948,1.753611,4108.472196,-1.872946,0.453422
44,21680.443060,1.845998,3784.294908,-1.898626,0.443211
60,20968.886797,1.825977,3601.300306,-1.919588,0.659967
59,20840.387997,1.822083,3569.540911,-1.884365,0.565866
62,21176.490729,1.900813,3654.370811,-1.881572,0.530832


A fin de ilustración se acompaña una gráfica que muestra la actividad original del conjunto de prueba (azul) contra la actividad obtenida con la predicción del modelo (rojo):

In [23]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[1], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[1], color='red', alpha=0.5)

show(p)

In [24]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[64], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[64], color='red', alpha=0.5)

show(p)

In [25]:
# create a new plot with a datetime axis type
p = figure(width=1000, 
           height=500, 
           x_axis_type="datetime", 
           x_axis_label='hora', 
           y_axis_label='conteos LOG', 
           title='Prediccion en el conjunto de prueba')

p.line(test.index, test[88], color='navy', alpha=0.5)
p.line(forecast_matrix.index, forecast_matrix[88], color='red', alpha=0.5)

show(p)

Como se dijo anteriormente, el modelo muestra cierta tendencia que corresponde al último conjunto de estacionalidad y media con el que se cuenta en el conjunto de entrenamiento, esto hace que la predicción tenga una tendencia positiva (o negativa) gradual a lo largo de las predicciones.